In [ ]:
import scipy.io
import numpy as np
import pandas as pd
import scanpy as sc
import h5py

import os
import numpy as np
import pandas as pd
import scanpy as sc
import anndata 
import seaborn as sns
from scipy.stats import zscore
import matplotlib.pyplot as plt
import collections
from natsort import natsorted

from scipy import stats
from scipy import sparse
from sklearn.decomposition import PCA
from umap import UMAP

from matplotlib.colors import LinearSegmentedColormap
from statsmodels.stats.multitest import multipletests

from scroutines.config_plots import *
from scroutines import powerplots # .config_plots import *
from scroutines import pnmf
from scroutines import basicu
from scroutines.gene_modules import GeneModules  

import atac_utils

In [ ]:
outdirfig = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/figures/250409'
!mkdir -p $outdirfig

# load data

In [ ]:
%%time
adata = anndata.read("/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/superdupermegaRNA_hasraw_multiome_l23.h5ad")
adata

In [ ]:
scores_abc = pd.read_csv("/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/scores_l23abc.csv", 
                         index_col=0,
                        )
scores_abc['scores_c-a'] = scores_abc['scores_c'] - scores_abc['scores_a']

adata.obs['scores_a'] = scores_abc.loc[adata.obs.index,'scores_a'].copy()
adata.obs['scores_b'] = scores_abc.loc[adata.obs.index,'scores_b'].copy()
adata.obs['scores_c'] = scores_abc.loc[adata.obs.index,'scores_c'].copy()
adata.obs['scores_c-a'] = scores_abc.loc[adata.obs.index,'scores_c-a'].copy()

# fix cond incompatibility # P6NR vs P6
adata.obs['cond'] = adata.obs['cond'].apply(lambda x: x.replace('NR', ''))

In [ ]:
condcode2cond = atac_utils.CONDCODE_TO_COND
condcode2cond

In [ ]:
sample_conditions = np.array(list(condcode2cond.values()))
n_cond = len(sample_conditions)

nr_condcodes  = np.array([0,1,2,3,5,7,9])
dr_condcodes  = np.array([4,6,8,10])

nr_conditions = sample_conditions[nr_condcodes]
dr_conditions = sample_conditions[dr_condcodes]

nr_times = [6,8,10,12,14,17,21]
dr_times =        [12,14,17,21]
times = nr_times

print(sample_conditions)
print(nr_condcodes, nr_conditions)
print(dr_condcodes, dr_conditions)

In [ ]:
%%time
adatas_pk = []
for cond in sample_conditions:
    print(cond)
    f = f'/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/results_atac/pmat_l23concensus_{cond}.h5ad'
    adata_pk = sc.read(f)
    adatas_pk.append(adata_pk)
    print(adata_pk)
    # break

In [ ]:
n_cond = len(sample_conditions)
n_peak = adatas_pk[0].shape[1]
n_type = 5

# DAR info

In [ ]:
# %%time
fin1 = '/u/home/f/f7xiesnm/v1_multiome/DAR_l23abc_qs_250411.npy'
fin2 = '/u/home/f/f7xiesnm/v1_multiome/DAR_l23abc_l2fc_250411.npy'
# fin3 = '/u/home/f/f7xiesnm/v1_multiome/DAR_l23abc_cpm_tensor_250411.npy'

qs_tensor   = np.load(fin1)
l2fc_tensor = np.load(fin2)

qs_tensor.shape, l2fc_tensor.shape

In [ ]:
l2fc_th = np.log2(2)
l2fc_th_s = np.log2(1.2)
alpha_th = 0.05

In [ ]:
qs_ca   = qs_tensor[:,0,:]
qs_ba   = qs_tensor[:,1,:]
qs_bc   = qs_tensor[:,2,:]

l2fc_ca = l2fc_tensor[:,0,:]
l2fc_ba = l2fc_tensor[:,1,:]
l2fc_bc = l2fc_tensor[:,2,:]

In [ ]:
qs_a   = np.minimum(qs_ca, qs_ba) # the better of the two
qs_c   = np.minimum(qs_ca, qs_bc) # the better of the two
qs_b   = np.minimum(qs_ba, qs_bc) # the better of the two

l2fc_a = np.max([-l2fc_ca, -l2fc_ba], axis=0) # mean fold change
l2fc_c = np.max([ l2fc_ca, -l2fc_bc], axis=0) # mean fold change
l2fc_b = np.max([ l2fc_ba,  l2fc_bc], axis=0) # mean fold change


l2fc_as = np.min([-l2fc_ca, -l2fc_ba], axis=0) # mean fold change
l2fc_cs = np.min([ l2fc_ca, -l2fc_bc], axis=0) # mean fold change
l2fc_bs = np.min([ l2fc_ba,  l2fc_bc], axis=0) # mean fold change


cond_sig_a = np.all([-l2fc_ca > 0, -l2fc_ba > 0, l2fc_a > l2fc_th, qs_a < alpha_th], axis=0)
cond_sig_c = np.all([ l2fc_ca > 0, -l2fc_bc > 0, l2fc_c > l2fc_th, qs_c < alpha_th], axis=0)
# cond_sig_a = np.all([-l2fc_ca > 0, -l2fc_ba > 0, l2fc_a > l2fc_th, qs_a < alpha_th, l2fc_as > l2fc_th_s], axis=0)
# cond_sig_c = np.all([ l2fc_ca > 0, -l2fc_bc > 0, l2fc_c > l2fc_th, qs_c < alpha_th, l2fc_cs > l2fc_th_s], axis=0)
cond_sig_b = np.all([ l2fc_ba > 0,  l2fc_bc > 0, l2fc_b > l2fc_th, qs_b < alpha_th, l2fc_bs > l2fc_th_s], axis=0)


instances, counts_a = np.unique(cond_sig_a.sum(axis=0), return_counts=True)
instances, counts_c = np.unique(cond_sig_c.sum(axis=0), return_counts=True)
instances, counts_b = np.unique(cond_sig_b.sum(axis=0), return_counts=True)

print('num A genes for each cond:\t', cond_sig_a.sum(axis=1), np.any(cond_sig_a, axis=0).sum())
print('num C genes for each cond:\t', cond_sig_c.sum(axis=1), np.any(cond_sig_c, axis=0).sum())
print('num B genes for each cond:\t', cond_sig_b.sum(axis=1), np.any(cond_sig_b, axis=0).sum())

print('num A genes in num conds:\t',  counts_a[1:])
print('num C genes in num conds:\t',  counts_c[1:])
print('num B genes in num conds:\t',  counts_b[1:])

# single-cell embeddings

In [ ]:
%%time
cond = 'P21'
cond_order = 9

# ref
adatasub = adata[adata.obs['cond']==cond]
cells_rna = adatasub.obs.index.values

# get ATAC
adata_pk = adatas_pk[cond_order]
cells_atac = adata_pk.obs.index.values

cells_mult = np.intersect1d(cells_rna, cells_atac)

cellcov = np.array(adata_pk[cells_mult].X.sum(axis=1)) #.reshape(-1,)

# 
mat = np.array(adata_pk[cells_mult].X.todense()) 
# mat = np.array(adata_pk[cells_mult, cond_any_sig].X.todense()) 

# size norm (CPM) between cells
mat = np.log2(mat/cellcov*1e4 + 1)

# ATAC PCS
atac_pca = PCA(n_components=3)
atac_pcs = atac_pca.fit_transform(mat)
    
# check this works
V = atac_pca.components_.T
assert np.all((mat-np.mean(mat, axis=0)).dot(V) - atac_pcs < 1e-2)

In [ ]:
%%time
cond = 'P8'
cond_order = 1

# ref
adatasub = adata[adata.obs['cond']==cond]
cells_rna = adatasub.obs.index.values

# get ATAC
adata_pk = adatas_pk[cond_order]
cells_atac = adata_pk.obs.index.values

cells_mult = np.intersect1d(cells_rna, cells_atac)

cellcov = np.array(adata_pk[cells_mult].X.sum(axis=1)) #.reshape(-1,)

# 
mat = np.array(adata_pk[cells_mult].X.todense()) 
# mat = np.array(adata_pk[cells_mult, cond_any_sig].X.todense()) 

# size norm (CPM) between cells
mat = np.log2(mat/cellcov*1e4 + 1)

# ATAC PCS
atac_pca_early = PCA(n_components=3)
atac_pcs_early = atac_pca_early.fit_transform(mat)
    
# check this works
V_early = atac_pca_early.components_.T
assert np.all((mat-np.mean(mat, axis=0)).dot(V_early) - atac_pcs_early < 1e-2)

# same set of peaks

In [ ]:
condcode2cond

In [ ]:
%%time

all_res = {}

for cond_order, cond in condcode2cond.items():
#     # get sub
    # get sub
    adatasub = adata[adata.obs['cond']==cond]
    cells_rna = adatasub.obs.index.values

    # get ATAC
    adata_pk = adatas_pk[cond_order]
    cells_atac = adata_pk.obs.index.values
    
    # overlapping cells
    cells_mult = np.intersect1d(cells_rna, cells_atac)
    print(cond, len(cells_mult)/len(cells_rna), len(cells_mult)/len(cells_atac), len(cells_atac))


    # atac counts
    cellcov = np.array(adata_pk[cells_mult].X.sum(axis=1)).reshape(-1,)
    # thresholding
    cell_filter = cellcov > 1e3
    
    cells_mult = cells_mult[cell_filter]
    cellcov = cellcov[cell_filter]
    
    # rna embedding
    rna_meta = adatasub[cells_mult].obs
    rna_scores_a = rna_meta['scores_a'].values
    rna_scores_b = rna_meta['scores_b'].values
    rna_scores_c = rna_meta['scores_c'].values
    
    # atac embedding
    mat = np.array(adata_pk[cells_mult].X.todense()) 

    # size norm (CP10k) between cells
    mat = np.log2(mat/cellcov.reshape(-1,1)*1e4 + 1)
    
    # region abc scores
    cond_pt_a = cond_sig_a[cond_order]
    cond_pt_b = cond_sig_b[cond_order]
    cond_pt_c = cond_sig_c[cond_order]

    atac_scores_a = np.mean(mat[:,cond_pt_a], axis=1)
    atac_scores_b = np.mean(mat[:,cond_pt_b], axis=1)
    atac_scores_c = np.mean(mat[:,cond_pt_c], axis=1)
    
    # region abc scores (P21)
    cond_p21_a = cond_sig_a[9]
    cond_p21_b = cond_sig_b[9]
    cond_p21_c = cond_sig_c[9]

    atac_p21scores_a = np.mean(mat[:,cond_p21_a], axis=1)
    atac_p21scores_b = np.mean(mat[:,cond_p21_b], axis=1)
    atac_p21scores_c = np.mean(mat[:,cond_p21_c], axis=1)
    
    # ATAC PCS
    # de novo
    # atac_pcs = PCA(n_components=5).fit_transform(mat)
    # projected to P21
    
    atac_pcs = (mat-np.mean(mat, axis=0)).dot(V)
    atac_pcs_early = (mat-np.mean(mat, axis=0)).dot(V_early)
    
    # all_rna_pcs[cond] = rna_pcs
    all_res[cond] = {
        'rna_scores_a':  rna_scores_a,
        'rna_scores_b':  rna_scores_b,
        'rna_scores_c':  rna_scores_c,
        
        'atac_p21scores_a': atac_p21scores_a,
        'atac_p21scores_b': atac_p21scores_b,
        'atac_p21scores_c': atac_p21scores_c,
        
        'atac_pcs': atac_pcs,
        'atac_pcs_early': atac_pcs_early,
        'atac_mat': mat,
    }


In [ ]:
all_atac_p21scores_cma = []
all_atac_p21scores_b = []

ac_mins = []
ac_maxs = []

b_mins = []
b_maxs = []

p_min, p_max = 2, 98

for cond in ['P6', 'P8', 'P10', 'P12', 'P14', 'P17', 'P21']:
    res = all_res[cond]
    ac_scores = res['atac_p21scores_c']-res['atac_p21scores_a']
    b_scores  = res['atac_p21scores_b']
    
    ac_mins.append(np.percentile(ac_scores, p_min))
    b_mins.append(np.percentile(b_scores, p_min))
    
    ac_maxs.append(np.percentile(ac_scores, p_max))
    b_maxs.append(np.percentile(b_scores, p_max))
    
    all_atac_p21scores_cma.append(ac_scores)
    all_atac_p21scores_b.append(b_scores)
    
all_atac_p21scores_cma = np.hstack(all_atac_p21scores_cma)
all_atac_p21scores_b   = np.hstack(all_atac_p21scores_b)


vmin_cma = np.percentile(all_atac_p21scores_cma, p_min)
vmax_cma = np.percentile(all_atac_p21scores_cma, p_max)

vmin_b = np.percentile(all_atac_p21scores_b, p_min)
vmax_b = np.percentile(all_atac_p21scores_b, p_max)

vmins_v2 = [vmin_cma, vmin_b]
vmaxs_v2 = [vmax_cma, vmax_b]

In [ ]:
from matplotlib.colors import LinearSegmentedColormap

colors_a = [(0.0, 'white'), (1.0, 'C0')]      
colors_b = [(0.0, 'white'), (1.0, 'C1')]      
colors_c = [(0.0, 'white'), (1.0, 'C2')]      
colors_n = [(0.0, 'white'), (1.0, 'lightgray')]      
cmap_a = LinearSegmentedColormap.from_list('cmap_a', colors_a)
cmap_b = LinearSegmentedColormap.from_list('cmap_b', colors_b)
cmap_c = LinearSegmentedColormap.from_list('cmap_c', colors_c)
cmap_n = LinearSegmentedColormap.from_list('cmap_n', colors_n)

colors_ac = [
    np.array(cmap_a(1.0)),
    0.5*np.array(cmap_a(1.0))+0.5*np.array(cmap_n(1.0)),
    np.array(cmap_n(1.0)),
    0.5*np.array(cmap_c(1.0))+0.5*np.array(cmap_n(1.0)),
    np.array(cmap_c(1.0)),
]
cmap_ac = LinearSegmentedColormap.from_list('cmap_ac', colors_ac)

colors_b2 = [
    'lightgray',
    cmap_b(0.2),
    cmap_b(1.0),
]
cmap_b2 = LinearSegmentedColormap.from_list('cmap_b2', colors_b2)

In [ ]:
cmaps = [cmap_ac, cmap_b2]

fig, axs = plt.subplots(2, 7, figsize=(7*3, 2*3), sharex=True, sharey=True)
j = 0
for cond_order, cond in condcode2cond.items():
    if cond.endswith('DR'):
        continue 
        
    res = all_res[cond]
    atac_pcs = res['atac_pcs']
    vecs = [
        res['atac_p21scores_c']-res['atac_p21scores_a'],
        res['atac_p21scores_b'],
    ]

    for i in range(2):
        ax = axs[i, j]
        vec = vecs[i]
        vmin_i = [ac_mins[2], b_mins[2]][i] # vmins_v2[i] #np.percentile(vec,  2)
        vmax_i = [ac_maxs[2], b_maxs[2]][i] #np.percentile(vec, 98)
        ax.scatter(-atac_pcs[:,1], atac_pcs[:,2], 
                   c=vec,
                   cmap=cmaps[i], # 'coolwarm',
                   vmin=vmin_i,
                   vmax=vmax_i,
                   rasterized=True,
                   edgecolor='none',
                   s=5)
        
        sns.despine(ax=ax)
        ax.grid(False)
        ax.set_aspect('equal')
        ax.set_xticks([])
        ax.set_yticks([])
        
        if i == 0:
            ax.set_title(cond)
    j += 1
    
output = os.path.join(outdirfig, f'pc23_atac_triangle_p10scale.pdf')
powerplots.savefig_autodate(fig, output)
plt.show()

In [ ]:
cmaps = [cmap_ac, cmap_b2]

fig, axs = plt.subplots(2, 7, figsize=(7*3, 2*3), sharex=True, sharey=True)
j = 0
for cond_order, cond in condcode2cond.items():
    if cond.endswith('DR'):
        continue 
        
    res = all_res[cond]
    atac_pcs = res['atac_pcs']
    vecs = [
        res['atac_p21scores_c']-res['atac_p21scores_a'],
        res['atac_p21scores_b'],
    ]

    for i in range(2):
        ax = axs[i, j]
        vec = vecs[i]
        vmin_i = [ac_mins[6], b_mins[6]][i] # vmins_v2[i] #np.percentile(vec,  2)
        vmax_i = [ac_maxs[6], b_maxs[6]][i] #np.percentile(vec, 98)
        ax.scatter(-atac_pcs[:,1], atac_pcs[:,2], 
                   c=vec,
                   cmap=cmaps[i], # 'coolwarm',
                   vmin=vmin_i,
                   vmax=vmax_i,
                   rasterized=True,
                   # edgecolor='none',
                   s=1)
        
        sns.despine(ax=ax)
        ax.grid(False)
        ax.set_aspect('equal')
        ax.set_xticks([])
        ax.set_yticks([])
        
        if i == 0:
            ax.set_title(cond)
    j += 1
    
output = os.path.join(outdirfig, f'pc23_atac_triangle_p21scale.pdf')
powerplots.savefig_autodate(fig, output)
plt.show()

In [ ]:
cmaps = [cmap_ac, cmap_b2]

fig, axs = plt.subplots(2, 7, figsize=(7*3, 2*3), sharex=True, sharey=True)
j = 0
for cond_order, cond in condcode2cond.items():
    if cond.endswith('DR'):
        continue 
        
    res = all_res[cond]
    atac_pcs = res['atac_pcs']
    vecs = [
        res['atac_p21scores_c']-res['atac_p21scores_a'],
        res['atac_p21scores_b'],
    ]

    for i in range(2):
        ax = axs[i, j]
        vec = vecs[i]
        vmin_i = vmins_v2[i] #np.percentile(vec,  2)
        vmax_i = vmaxs_v2[i] #np.percentile(vec, 98)
        ax.scatter(-atac_pcs[:,1], atac_pcs[:,2], 
                   c=vec,
                   cmap=cmaps[i], # 'coolwarm',
                   vmin=vmin_i,
                   vmax=vmax_i,
                   rasterized=True,
                   # edgecolor='none',
                   s=1)
        
        sns.despine(ax=ax)
        ax.grid(False)
        ax.set_aspect('equal')
        ax.set_xticks([])
        ax.set_yticks([])
        
        if i == 0:
            ax.set_title(cond)
    j += 1
    
output = os.path.join(outdirfig, f'pc23_atac_triangle_wholescale.pdf')
powerplots.savefig_autodate(fig, output)
plt.show()

In [ ]:
cmaps = [cmap_ac, cmap_b2]

fig, axs = plt.subplots(2, 7, figsize=(7*3, 2*3), sharex=True, sharey=True)
j = 0
for cond_order, cond in condcode2cond.items():
    if cond.endswith('DR'):
        continue 
        
    res = all_res[cond]
    atac_pcs = res['atac_pcs']
    vecs = [
        res['atac_p21scores_c']-res['atac_p21scores_a'],
        res['atac_p21scores_b'],
    ]

    for i in range(2):
        ax = axs[i, j]
        vec = vecs[i]
        vmin_i = np.percentile(vec,  2)
        vmax_i = np.percentile(vec, 98)
        ax.scatter(-atac_pcs[:,1], atac_pcs[:,2], 
                   c=vec,
                   cmap=cmaps[i], # 'coolwarm',
                   vmin=vmin_i,
                   vmax=vmax_i,
                   rasterized=True,
                   s=1)
        
        sns.despine(ax=ax)
        ax.grid(False)
        ax.set_aspect('equal')
        ax.set_xticks([])
        ax.set_yticks([])
        
        if i == 0:
            ax.set_title(cond)
    j += 1
    
output = os.path.join(outdirfig, f'pc23p21_atac_triangle.pdf')
powerplots.savefig_autodate(fig, output)
plt.show()


In [ ]:
cmaps = [cmap_ac, cmap_b2]

fig, axs = plt.subplots(2, 7, figsize=(7*3, 2*3), sharex=True, sharey=True)
j = 0
for cond_order, cond in condcode2cond.items():
    if cond.endswith('DR'):
        continue 
        
    res = all_res[cond]
    atac_pcs = res['atac_pcs_early']
    vecs = [
        res['atac_p21scores_c']-res['atac_p21scores_a'],
        res['atac_p21scores_b'],
    ]

    for i in range(2):
        ax = axs[i, j]
        vec = vecs[i]
        vmin_i = vmins_v2[i] #np.percentile(vec,  2)
        vmax_i = vmaxs_v2[i] #np.percentile(vec, 98)
        ax.scatter(atac_pcs[:,1], atac_pcs[:,2], 
                   c=vec,
                   cmap=cmaps[i], # 'coolwarm',
                   vmin=vmin_i,
                   vmax=vmax_i,
                   rasterized=True,
                   s=1)
        
        sns.despine(ax=ax)
        ax.grid(False)
        ax.set_aspect('equal')
        ax.set_xticks([])
        ax.set_yticks([])
        
        if i == 0:
            ax.set_title(cond)
    j += 1
output = os.path.join(outdirfig, f'pc23_p8_atac_triangle.pdf')
powerplots.savefig_autodate(fig, output)
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(3*3, 1*3), sharex=True, sharey=True)
cond_order = 9
cond = 'P21'
res  = all_res[cond]
atac_pcs = res['atac_pcs']
titles = ['a', 'b', 'c']
vecs = [res[f'atac_p21scores_{title}'] for title in titles]

for ax, vec, title in zip(axs, vecs, titles):
    vmin = np.percentile(vec,  2)
    vmax = np.percentile(vec, 98)
    ax.scatter(-atac_pcs[:,1], atac_pcs[:,2], 
               c=vec,
               cmap='coolwarm',
               vmin=vmin, 
               vmax=vmax,
               rasterized=True,
               s=1)
    # ax.set_title(cond)
    sns.despine(ax=ax)
    ax.grid(False)
    ax.set_aspect('equal')
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title(title)
output = os.path.join(outdirfig, f'pc23_atac_triangle_p21.pdf')
powerplots.savefig_autodate(fig, output)
plt.show()


In [ ]:
vec.shape